 Imports and Setup

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
rouge = evaluate.load("rouge")

Helper Functions

In [14]:
def generate_summary(model, tokenizer, text, device, max_length=128):
    input_ids = tokenizer.encode(
        text, return_tensors="pt", truncation=True, max_length=512
    ).to(device)
    outputs = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

def evaluate_summary(pred, ref):
    scores = rouge.compute(predictions=[pred], references=[ref])
    return {k: round(v * 100, 2) for k, v in scores.items()}

def load_model(base_model_name, repo_id, device):
    base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name).to(device)
    if repo_id:
        try:
            model = PeftModel.from_pretrained(base_model, repo_id).to(device)
            print(f"Loaded LoRA adapter from {repo_id}")
            return model
        except Exception as e:
            print(f"Error loading LoRA adapter from {repo_id}: {e}. Using base model instead.")
            return base_model
    else:
        print("Using baseline pretrained model")
        return base_model

Load Models

In [15]:
model_repos = {
    "Baseline (Pretrained)": None,
    "LoRA Local": "benitoals/my-lora",
    "HF Science Dataset": "benitoals/my-lora-hf",
    "HF + Local Fine-tuned": "benitoals/my-lora-local-combined"
}

models = {}

for model_label, repo_id in model_repos.items():
    print(f"\nLoading {model_label}...")
    models[model_label] = load_model(model_name, repo_id, device).eval()


Loading Baseline (Pretrained)...
Using baseline pretrained model

Loading LoRA Local...
Loaded LoRA adapter from benitoals/my-lora

Loading HF Science Dataset...
Loaded LoRA adapter from benitoals/my-lora-hf

Loading HF + Local Fine-tuned...
Loaded LoRA adapter from benitoals/my-lora-local-combined


Interactive Evaluation

In [16]:
input_text = input("Enter the article text to summarize:\n").strip()
reference_summary = input("Enter the reference abstract:\n").strip()

for model_label, model in models.items():
    print(f"\n=== {model_label} ===")
    summary = generate_summary(model, tokenizer, input_text, device)
    scores = evaluate_summary(summary, reference_summary)

    print("\nGenerated Summary:\n", summary)
    print("\nReference Abstract:\n", reference_summary)
    print("\nROUGE Scores:", scores)
    print("-" * 70)


=== Baseline (Pretrained) ===

Generated Summary:
 <extra_id_0> could interfere.

Reference Abstract:
 Formal Veriﬁcation of Ladder Logic programs using NuSMV Sam Kottler, Mehdy Khayamy,ySyed Rafay Hasan,zand Omar Elkeelanyx Colorado College, Department of Mathematics and Computer Science, sam.kottler@coloradocollege.edu y z xTennessee Technological University, Electrical and Computer Engineering Department ymkhayamy42@students.tntech.edu,zshasan@tntech.edu,xoelkeelany@tntech.edu Abstract —Programmable logic controllers (PLCs) are heavy- duty computers used to control industrial systems. For many years these systems were physically separated from any other network making attacks extremely difﬁcult. However, these in- creasingly connected systems have not improved much in terms of security, leaving them vulnerable to attacks. This paper attempts to show that ladder logic programs for PLCs can be modeled in NuSMV and veriﬁed using computational tree logic (CTL) speciﬁcations. This pap